In [37]:
# Header block to include all modules that must be imported ahead of time
# Only needs to be run once per session, and each time a new module is added

import requests
import json
import pandas as pd # this will need to be installed via command line first
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [38]:
# Method to get all the revelent player attributes for every player currently signed on every NFL team
# The CBS Sports website is used as the source

def GetPlayterAttributes(team: list): # accepts a list ['team-name', 'LOC']
    # Use requests to get the raw HTML response from cbssports.com
    CBSResponse = requests.get(f'https://www.cbssports.com/nfl/teams/{team[1]}/{team[0]}/roster/')
    
    # If we get a normal response proceed with data scraping
    if CBSResponse.status_code == 200:
                 
        # Confirm the request was successful
        print(f"Request succeeded with status code {CBSResponse.status_code}")
        
        # Use Pandas to read the HTML content and put it into a DataFrame
        TableTag = 'TableBase-table'
        DataFrames = pd.read_html(CBSResponse.content, attrs={'class': TableTag})
        
        # Select the relevant data from the DataFrame(offense, defense, and special teams)
        OffenseDF = DataFrames[0][['Player', 'POS', 'HT', 'WT', 'DOB (AGE)', 'EXP']]
        DefenseDF = DataFrames[1][['Player', 'POS', 'HT', 'WT', 'DOB (AGE)', 'EXP']]
        SpecialDF = DataFrames[2][['Player', 'POS', 'HT', 'WT', 'DOB (AGE)', 'EXP']]
        
        # Combine the three DataFrames into one
        FullTeamDF = pd.concat([OffenseDF, DefenseDF, SpecialDF], axis = 0, ignore_index = True)
        # print(FullTeamDF.head(20))
        
        # Method to clean the player names since CBS puts player news directly in the name cell
        def CleanNameCell(NameText):
            # Split the name cell into a list
            NameText = NameText.split(' ')
            
            # If the name cell doesn't contain player news and no name postfix
            if len(NameText) == 3:
                # Remove the first initial by selecting the rest of the elements
                NameText = NameText[1:]
                # Extract the first name from the first-last name string
                NameText[0] = NameText[0][len(NameText[1]):]
                # Create new string with first and last name
                NameText = ' '.join(NameText)
                
            # The name cell contains a name postfix
            elif len(NameText) == 5:
                # Create new list to store the frist name, last name, postfix
                NameTextPostfix = []
                # Extract the first name from the first-suffix name string, add it to the list
                NameTextPostfix.append(NameText[2][len(NameText[4]):])
                # Add the last name to the list
                NameTextPostfix.append(NameText[1])
                # Add the postfix to the list
                NameTextPostfix.append(NameText[4])
                # Turn the list into a string
                NameText = ' '.join(NameTextPostfix)

            # Otherwise the player name cell contains player news, injury status
            else:
                # Select only the capitilized words from the player cell
                NameTextCapitalized = [s for s in NameText if s and not s[0].islower()]
                # Select the first inital which is always the first item in NameText when there is news, injury status
                First = NameTextCapitalized[0][0]
                # Select the last which is always the second item in NameText when there is news, injury status
                LastName =  NameTextCapitalized[1]
                # Create postfix varaible in case player name has it
                Postfix = ''
                PostfixList = {'Jr.', 'II','III', 'IV'} # set of possible postfixes

                # Iterate through the items in NameTextCapitalzied from the third item to the end
                for s in range(3, (len(NameTextCapitalized) - 1)):
                    # Make sure we're always comparing to the first letter of current item
                    FirstTemp = NameTextCapitalized[s][0]
                    # If the current element first char is the first initial and the next element is last name
                    if NameTextCapitalized[s][0] == FirstTemp and NameTextCapitalized[s + 1].replace(":", "") == LastName:
                        # If we haven't reached the second to last elementand the next next element is a postfix
                        if NameTextCapitalized[s + 2] in PostfixList:
                            # Set the postfix variable to the next next element
                            Postfix = NameTextCapitalized[s + 2].replace(".", "") # remove the period from the postfix

                        # Set the first and last name 
                        First = NameTextCapitalized[s]
                        LastName = NameTextCapitalized[s + 1].replace(":", "") # remove the possible colon from the last name
                        # No break point because the first instance of the full name might not contain a postfix
                        
                        
                # Recreats NameText with the first name, last name
                NameText = [First, LastName]
                # If a postfix was found, add it to the list
                if Postfix:
                    NameText.append(Postfix)
                
                NameText = ' '.join(NameText)

            
            return NameText
        

        # Apply the method to the player names column
        FullTeamDF['Player'] = FullTeamDF['Player'].apply(CleanNameCell)       
        
        # Capitilize the team name for consistency
        TeamNameCapitalized = (team[0].replace("-", " ")).split(' ')
        # We have to account the 49ers and their team name starting with a number
        TeamNameCapitalized = [s.capitalize() if not s[0].isdigit() else s for s in TeamNameCapitalized]
        TeamNameCapitalized = ' '.join(TeamNameCapitalized)
        
        # Create a .csv file with all the player names and their cap hits
        FullTeamDF.to_csv(f'./data/player-attributes-lists/{TeamNameCapitalized}-Player-Attribute-List.csv', index = False, header = False)
        
        # Read the csv back for testing purposes
        FullTeamCSV = pd.read_csv(f'./data/player-attributes-lists/{TeamNameCapitalized}-Player-Attribute-List.csv', header = None)
        FullTeamCSV.columns = ['Player', 'POS', 'HT', 'WT', 'DOB (AGE)', 'EXP']
        print(FullTeamCSV)
        

    # If we don't get a normal reponse, stop scraping
    else:
        print(f"Request failed with status code {CBSResponse.status_code}")
        return -1
    
GetPlayterAttributes(['buffalo-bills', 'BUF'])

Request succeeded with status code 200
                      Player  POS    HT   WT        DOB (AGE) EXP
0                 Josh Allen   QB   6-5  237   5/21/1996 (28)   7
1             Shane Buechele   QB   6-0  210    1/8/1998 (26)   3
2             Mitch Trubisky   QB   6-3  222   8/20/1994 (29)   8
3                 James Cook   RB  5-11  190   9/25/1999 (24)   3
4                  Ray Davis   RB  5-10  216  11/11/1999 (24)   R
5            Darrynton Evans   RB  5-10  203    7/9/1998 (26)   5
6             Frank Gore Jr.   RB   5-8  195   3/13/2002 (22)   R
7                 Ty Johnson   RB  5-10  210   9/17/1997 (26)   6
8             Reggie Gilliam   FB   6-0  244   8/20/1997 (26)   5
9             Chase Claypool   WR   6-4  238    7/7/1998 (26)   5
10              Keon Coleman   WR   6-4  215   5/17/2003 (21)   R
11               K.J. Hamler   WR   5-9  178    7/8/1999 (25)   4
12              Mack Hollins   WR   6-4  221   9/16/1993 (30)   8
13             Andy Isabella   WR   5

In [39]:
# Method to get the revelent attributes for every player currently signed on every NFL team
# Reads in the team list csv, and calls the GetTeamSalaryCapHits method for each team
def GetAllPlayerAttributes():
    # Read in the full team list csv
    FullTeamListCSV = pd.read_csv('./data/team-attribute-lists/Teams-List.csv', header = None)
    FullTeamListCSV.columns = ['TeamName', 'Abbreviation', 'Conference', 'Division']
    
    # Create a list of all the team names and abbreviations as a list of lists
    TeamList = []
    # Take the first two columns of each row, convert to a list, append to TeamList
    FullTeamListCSV.apply(lambda row: TeamList.append(row[:2].tolist()), axis = 1)
    
    # Get the salary cap hits for each team
    for team in TeamList:
        # format the team name to be used in the URL
        team[0] = team[0].replace(" ", "-").lower()
        GetPlayterAttributes(team)
    

GetAllPlayerAttributes()

Request succeeded with status code 200
                     Player  POS    HT   WT        DOB (AGE) EXP
0              Kyler Murray   QB  5-10  207    8/7/1997 (26)   6
1            Desmond Ridder   QB   6-3  207   8/31/1999 (24)   3
2              Clayton Tune   QB   6-3  220   3/23/1999 (25)   2
3               Trey Benson   RB   6-1  221   7/23/2002 (21)   R
4            Michael Carter   RB   5-8  201    5/7/1999 (25)   4
5              James Conner   RB   6-1  233    5/5/1995 (29)   8
6             DeeJay Dallas   RB  5-10  214   9/16/1998 (25)   5
7           Emari Demercado   RB   5-9  215   1/20/1999 (25)   2
8                Tony Jones   RB  5-11  224  11/24/1997 (26)   4
9              Daniel Arias   WR   6-3  216    9/2/1998 (25)   1
10          Andre Baccellia   WR  5-10  175    1/7/1997 (27)   2
11              Dan Chisena   WR   6-3  202   2/25/1997 (27)   3
12              Greg Dortch   WR   5-7  175   5/29/1998 (26)   4
13       Marvin Harrison Jr   WR   6-4  205   8/11/

Request succeeded with status code 200
                     Player  POS    HT   WT        DOB (AGE) EXP
0             Lamar Jackson   QB   6-2  215    1/7/1997 (27)   7
1              Josh Johnson   QB   6-3  214   5/15/1986 (38)  10
2               Emory Jones   QB   6-2  210   4/29/2000 (24)   R
3               Devin Leary   QB   6-1  215   9/10/1999 (24)   R
4               Rasheen Ali   RB  5-11  197   2/16/2001 (23)   R
5             Chris Collier   RB  5-11  203   3/24/2000 (24)   R
6             Derrick Henry   RB   6-3  247    1/4/1994 (30)   9
7              Justice Hill   RB  5-10  197  11/14/1997 (26)   6
8           Keaton Mitchell   RB   5-8  191   1/17/2002 (22)   2
9               Owen Wright   RB   5-9  217   2/28/1999 (25)   1
10           Patrick Ricard   FB   6-3  305   5/27/1994 (30)   8
11           Nelson Agholor   WR   6-0  200   5/24/1993 (31)  10
12           Rashod Bateman   WR   6-1  197  11/29/1999 (24)   4
13         Malik Cunningham   WR   6-1  198   10/6/

Request succeeded with status code 200
                  Player  POS    HT   WT        DOB (AGE) EXP
0            Andy Dalton   QB   6-2  220  10/29/1987 (36)  14
1           Jack Plummer   QB   6-5  215  10/21/1999 (24)   R
2            Bryce Young   QB  5-10  204   7/25/2001 (22)   2
3      Raheem Blackshear   RB   5-9  190    6/5/1998 (26)   3
4             Mike Boone   RB  5-10  206   7/30/1995 (28)   7
5        Jonathon Brooks   RB   6-0  207   7/21/2003 (20)   R
6          Chuba Hubbard   RB   6-1  210   6/11/1999 (25)   4
7          Rashaad Penny   RB  5-11  220    2/2/1996 (28)   7
8          Miles Sanders   RB  5-11  211    5/1/1997 (27)   6
9          Jaden Shirden   RB   5-9  195    5/2/2002 (22)   R
10           Jalen Coker   WR   6-3  213  10/30/2001 (22)   R
11         Daewood Davis   WR   6-2  195   1/23/1999 (25)   1
12       Diontae Johnson   WR  5-10  183    7/5/1996 (28)   6
13        Xavier Legette   WR   6-3  227   1/29/2001 (23)   R
14  Terrace Marshall Jr.   WR  

Request succeeded with status code 200
                 Player  POS    HT   WT        DOB (AGE) EXP
0         Jake Browning   QB   6-2  210   4/11/1996 (28)   2
1            Joe Burrow   QB   6-4  215  12/10/1996 (27)   5
2        Rocky Lombardi   QB   6-4  225   7/30/1998 (25)   R
3        Logan Woodside   QB   6-1  213   1/27/1995 (29)   4
4           Chase Brown   RB  5-10  211   3/21/2000 (24)   2
5             Noah Cain   RB  5-10  222  12/17/2000 (23)   R
6        Elijah Collins   RB   6-0  220    8/1/2000 (23)   R
7           Chris Evans   RB  5-11  215   10/5/1997 (26)   4
8             Zack Moss   RB  5-10  215  12/15/1997 (26)   5
9     Trayveon Williams   RB   5-8  206  10/18/1997 (26)   6
10         Cole Burgess   WR   6-1  192   2/11/2000 (24)   R
11      Jermaine Burton   WR   6-0  194   7/28/2001 (22)   R
12        Ja'Marr Chase   WR   6-0  201    3/1/2000 (24)   4
13          Tee Higgins   WR   6-4  219   1/18/1999 (25)   5
14       Andrei Iosivas   WR   6-3  205  10/15

Request succeeded with status code 200
                  Player  POS    HT   WT        DOB (AGE) EXP
0             Trey Lance   QB   6-4  226    5/9/2000 (24)   4
1           Dak Prescott   QB   6-2  238   7/29/1993 (30)   9
2            Cooper Rush   QB   6-3  225  11/21/1993 (30)   7
3           Snoop Conner   RB  5-10  222    8/1/2000 (23)   2
4            Malik Davis   RB  5-11  202  11/26/1998 (25)   2
5            Rico Dowdle   RB   6-0  215   6/14/1998 (26)   5
6        Ezekiel Elliott   RB   6-0  226   7/22/1995 (28)   9
7          Royce Freeman   RB   6-0  238   2/24/1996 (28)   6
8          Hunter Luepke   RB   6-1  238   2/28/2000 (24)   2
9         Nathaniel Peat   RB   5-8  197                —   R
10          Deuce Vaughn   RB   5-6  176   11/2/2001 (22)   2
11          Jalen Brooks   WR   6-2  205    5/7/2000 (24)   2
12         Brandin Cooks   WR  5-10  189   9/25/1993 (30)  11
13      Corey Crooms Jr.   WR  5-11  180                —   R
14          David Durden   WR  

...

Request succeeded with status code 200
                      Player  POS    HT   WT        DOB (AGE) EXP
0             Andrew Peasley   QB   6-2  218    1/6/2000 (24)   R
1              Aaron Rodgers   QB   6-2  223   12/2/1983 (40)  20
2               Tyrod Taylor   QB   6-1  217    8/3/1989 (34)  14
3              Jordan Travis   QB   6-1  200    5/2/2000 (24)   R
4          Israel Abanikanda   RB  5-10  216   10/5/2002 (21)   2
5              Braelon Allen   RB   6-1  235   1/20/2004 (20)   R
6                Nick Bawden   RB   6-2  245   6/22/1996 (28)   7
7                Tarik Cohen   RB   5-6  181   7/26/1995 (28)   5
8               Isaiah Davis   RB   6-1  220   2/21/2002 (22)   R
9                Breece Hall   RB  5-11  217   5/31/2001 (23)   3
10         Xazavian Valladay   RB   6-0  198    7/3/1998 (26)   1
11            Jason Brownlee   WR   6-3  202   1/30/1999 (25)   2
12             Irvin Charles   WR   6-4  219   4/13/1997 (27)   2
13            Malachi Corley   WR  5-

Request succeeded with status code 200
                   Player  POS    HT   WT        DOB (AGE) EXP
0              Kyle Allen   QB   6-3  210    3/8/1996 (28)   6
1           Justin Fields   QB   6-3  227    3/5/1999 (25)   4
2           J PlumleeJohn   QB   6-0  200    1/2/2001 (23)   R
3          Russell Wilson   QB  5-11  206  11/29/1988 (35)  13
4          Daijun Edwards   RB  5-10  201   4/11/2001 (23)   R
5            Najee Harris   RB   6-1  242    3/9/1998 (26)   4
6   Cordarrelle Patterson   RB   6-2  220   3/17/1991 (33)  12
7         La'Mical Perine   RB  5-11  216   1/30/1998 (26)   3
8         Aaron Shampklin   RB  5-10  195  10/18/1999 (24)   1
9           Jonathan Ward   RB   6-0  202   9/30/1997 (26)   5
10          Jaylen Warren   RB   5-8  215   11/1/1998 (25)   3
11          Jack Colletto   FB   6-3  237  11/19/1998 (25)   1
12      Calvin Austin III   WR   5-9  162   3/24/1999 (25)   3
13       Marquez Callaway   WR   6-2  204   3/27/1998 (26)   4
14         Jacob

Request succeeded with status code 200
                  Player  POS    HT   WT        DOB (AGE) EXP
0             Sam Howell   QB   6-1  220  11/16/2000 (23)   3
1             Geno Smith   QB   6-3  221  10/10/1990 (33)  12
2              PJ Walker   QB  5-11  210   2/26/1995 (29)   5
3        Zach Charbonnet   RB   6-1  214    1/8/2001 (23)   2
4          George Holani   RB  5-11  210  12/16/1999 (24)   R
5             Kobe Lewis   RB   5-9  208   6/14/2000 (24)   R
6         Kenny McIntosh   RB   6-0  204    3/3/2000 (24)   2
7        Ricky Person Jr   RB   6-0  217  11/12/1999 (24)   1
8     Kenneth Walker III   RB   5-9  211  10/20/2000 (23)   3
9              Jake Bobo   WR   6-4  207    8/4/1998 (25)   2
10      D'Wayne Eskridge   WR   5-9  190   3/23/1997 (27)   4
11         Hayden Hatten   WR   6-1  207    8/6/2000 (23)   R
12         Tyler Lockett   WR  5-10  182   9/28/1992 (31)  10
13            DK Metcalf   WR   6-4  235  12/14/1997 (26)   6
14  Laviska Shenault Jr.   WR  

Request succeeded with status code 200
                   Player  POS    HT   WT        DOB (AGE) EXP
0              Will Levis   QB   6-4  229   6/27/1999 (25)   2
1           Mason Rudolph   QB   6-5  235   6/17/1995 (29)   7
2            Malik Willis   QB   6-1  219   5/25/1999 (25)   3
3         Julius Chestnut   RB  5-11  228  10/26/2000 (23)   3
4          Hassan Haskins   RB   6-2  228  11/26/1999 (24)   3
5          Dillon Johnson   RB  5-11  217   6/15/2001 (23)   R
6            Tony Pollard   RB   6-0  215   4/30/1997 (27)   6
7            Jabari Small   RB   5-9  198   1/31/2002 (22)   R
8            Tyjae Spears   RB  5-11  195   6/15/2001 (23)   2
9              Tyler Boyd   WR   6-2  203  11/15/1994 (29)   9
10          Treylon Burks   WR   6-2  225   3/23/2000 (24)   3
11          Colton Dowell   WR   6-3  212   4/19/1999 (25)   2
12     Tre'Shaun Harrison   WR  5-11  188   6/26/2000 (24)   1
13        DeAndre Hopkins   WR   6-1  212    6/6/1992 (32)  11
14       Jha'Qua

"\n# A list of links to Over The Cap Salary Cap pages for various teams\nNFLTeams = [['buffalo-bills', 'BUF', 4], ['miami-dolphins', 'MIA', 20], ['new-england-patriots', 'NE0', 22], ['new-york-jets', 'NYJ', 25],\n            ['baltimore-ravens', 'BAL', 3], ['cincinnati-bengals', 'CIN', 7], ['cleveland-browns', 'CLE', 8], ['pittsburgh-steelers', 'PIT', 27],\n            ['houston-texans', 'HOU', 13], ['indianapolis-colts', 'IND', 14], ['jacksonville-jaguars', 'JAX', 15], ['tennessee-titans', 'TEN', 31], \n            ['denver-broncos', 'DEN', 10], ['kansas-city-chiefs', 'KC0', 16], ['las-vegas-raiders', 'LV0', 17], ['los-angeles-chargers', 'LAC', 18],\n            ['dallas-cowboys', 'DAL', 9], ['new-york-giants', 'NYG', 24], ['philadelphia-eagles', 'PHI', 26], ['washington-commanders', 'WSH', 32],\n            ['chicago-bears', 'CHI', 6], ['detroit-lions', 'DET', 11], ['green-bay-packers', 'GB0', 12], ['minnesota-vikings', 'MIN', 21],\n            ['atlanta-falcons', 'ATL', 2], ['caroli